In [1]:
# Cell 1: Reproducibility Setup
# =============================================================================
SEED = 5
import os, random
import numpy as np
import tensorflow as tf
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-09-03 07:11:40.699611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756883500.872422      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756883500.920529      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# =============================================================================
# Cell 2: Core Imports and Constants
# =============================================================================
import json, math, os, gc
import cv2
from PIL import Image
import pickle
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.utils import to_categorical

# Machine Learning
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# XAI Libraries
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
import seaborn as sns
from skimage.segmentation import mark_boundaries

import warnings
warnings.filterwarnings('ignore')

# Constants
IMG_SIZE = 224
BATCH_SIZE = 24
EPOCHS = 50
PATIENCE = 7

# GPU Memory Configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")

# Enable XLA compilation for better memory efficiency
tf.config.optimizer.set_jit(True)
print("✅ XLA compilation enabled for memory efficiency")

# Kaggle input paths
PROCESSED_DATA_PATH = "/kaggle/input/please/preprocessed_aptos_data.npz"
TEST_CSV_PATH = "/kaggle/input/please/test_data.csv"
TRAIN_CSV_PATH = "/kaggle/input/please/train_data.csv"

print("✅ All imports completed successfully")

✅ GPU memory growth enabled for 1 GPU(s)
✅ XLA compilation enabled for memory efficiency
✅ All imports completed successfully


In [3]:
# =============================================================================
# Cell 3: Load and Prepare Data
# =============================================================================
print("Loading preprocessed data...")

# Load preprocessed data
data = np.load(PROCESSED_DATA_PATH)
print("Available keys in preprocessed data:", list(data.keys()))

# Load data using the correct keys
x_train_full = data['X_train']
y_train_full = data['y_train']
x_test = data['X_test']  
y_test = data['y_test']

# Load CSV files for additional metadata
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

print(f"Training data shape: {x_train_full.shape}")
print(f"Training labels shape: {y_train_full.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")

# Convert labels to one-hot encoding if needed
if len(y_train_full.shape) == 1 or (len(y_train_full.shape) == 2 and y_train_full.shape[1] == 1):
    print("Converting training labels to one-hot encoding...")
    y_train_full = to_categorical(y_train_full, num_classes=5)
    print(f"New training labels shape: {y_train_full.shape}")

if len(y_test.shape) == 1 or (len(y_test.shape) == 2 and y_test.shape[1] == 1):
    print("Converting test labels to one-hot encoding...")
    y_test = to_categorical(y_test, num_classes=5)
    print(f"New test labels shape: {y_test.shape}")

# Normalize pixel values to [0,1] if needed
if x_train_full.max() > 1.0:
    print("Normalizing pixel values to [0,1]...")
    x_train_full = x_train_full.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
else:
    print("Data already normalized")
    x_train_full = x_train_full.astype('float32')
    x_test = x_test.astype('float32')

# Split training data into train/validation (80/20 split for single fold)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full, 
    test_size=0.2, 
    random_state=SEED, 
    stratify=np.argmax(y_train_full, axis=1)
)

print(f"Final data splits:")
print(f"  Training: {x_train.shape}")
print(f"  Validation: {x_val.shape}")
print(f"  Test: {x_test.shape}")

print("✅ Data loaded and prepared successfully")

Loading preprocessed data...
Available keys in preprocessed data: ['X_train', 'y_train', 'X_test', 'y_test']
Training data shape: (7220, 224, 224, 3)
Training labels shape: (7220, 5)
Test data shape: (733, 224, 224, 3)
Test labels shape: (733,)
Converting test labels to one-hot encoding...
New test labels shape: (733, 5)
Normalizing pixel values to [0,1]...
Final data splits:
  Training: (5776, 224, 224, 3)
  Validation: (1444, 224, 224, 3)
  Test: (733, 224, 224, 3)
✅ Data loaded and prepared successfully


In [4]:
# =============================================================================
# Cell 4: MixupGenerator Implementation
# =============================================================================
class MixupGenerator:
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen
        
    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))
            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2: (i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)
                yield X, y
                
    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)
        if self.shuffle:
            np.random.shuffle(indexes)
        return indexes
    
    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)
        
        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)
        
        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])
        
        y1 = self.y_train[batch_ids[:self.batch_size]]
        y2 = self.y_train[batch_ids[self.batch_size:]]
        y = y1 * y_l + y2 * (1 - y_l)
        
        return X, y

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,
        fill_mode='constant',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=True,
    )

print("✅ MixupGenerator defined")

✅ MixupGenerator defined


In [5]:
# Cell 5: DenseNet Architecture Definition
# =============================================================================
def build_densenet169():
    """
    Build DenseNet169 model with the same architecture pattern as other models
    """
    base_model = DenseNet169(
        include_top=False,
        weights='imagenet',
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')
    ])
    
    # Make all layers trainable
    for layer in model.layers:
        layer.trainable = True
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.00005),
        metrics=['accuracy']
    )
    
    return model

print("✅ DenseNet169 architecture defined")

✅ DenseNet169 architecture defined


In [6]:
# Cell 6: Single Fold Training Function
# =============================================================================
def train_single_fold_model(model_name, model_fn, x_train, y_train, x_val, y_val, x_test, y_test):
    """
    Train a single model without cross-validation
    """
    print(f"\n{'='*60}")
    print(f"Training {model_name} - Single Fold")
    print(f"{'='*60}")
    
    # Clear session and collect garbage
    tf.keras.backend.clear_session()
    gc.collect()
    
    print(f"Training samples: {len(x_train)}")
    print(f"Validation samples: {len(x_val)}")
    print(f"Test samples: {len(x_test)}")
    
    # Build model
    model = model_fn()
    print(f"Model built with {model.count_params():,} parameters")
    
    # Setup callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=PATIENCE,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.5,
            patience=3,
            min_lr=1e-8,
            verbose=1
        )
    ]
    
    # Setup data generators
    datagen = create_datagen()
    mixup_gen = MixupGenerator(x_train, y_train, batch_size=BATCH_SIZE, datagen=datagen)
    
    # Calculate steps per epoch
    steps_per_epoch = len(x_train) // (BATCH_SIZE * 2)
    print(f"Steps per epoch: {steps_per_epoch}, Batch size: {BATCH_SIZE}")
    
    # Train model
    try:
        history = model.fit(
            mixup_gen(),
            steps_per_epoch=steps_per_epoch,
            epochs=EPOCHS,
            validation_data=(x_val, y_val),
            validation_batch_size=BATCH_SIZE,
            callbacks=callbacks,
            verbose=1
        )
    except tf.errors.ResourceExhaustedError:
        print("⚠️ OOM error, trying with smaller batch size...")
        smaller_batch = BATCH_SIZE // 2
        mixup_gen_small = MixupGenerator(x_train, y_train, batch_size=smaller_batch, datagen=datagen)
        steps_per_epoch_small = len(x_train) // (smaller_batch * 2)
        
        history = model.fit(
            mixup_gen_small(),
            steps_per_epoch=steps_per_epoch_small,
            epochs=EPOCHS,
            validation_data=(x_val, y_val),
            validation_batch_size=smaller_batch,
            callbacks=callbacks,
            verbose=1
        )
        print(f"✅ Training completed with reduced batch size: {smaller_batch}")
    
    # Evaluate on validation and test sets
    val_loss, val_acc = model.evaluate(x_val, y_val, batch_size=BATCH_SIZE, verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=0)
    
    # Get predictions for detailed metrics
    y_val_pred = model.predict(x_val, batch_size=BATCH_SIZE, verbose=0)
    y_test_pred = model.predict(x_test, batch_size=BATCH_SIZE, verbose=0)
    
    # Convert predictions and true labels
    y_val_true_labels = np.argmax(y_val, axis=1)
    y_val_pred_labels = np.argmax(y_val_pred, axis=1)
    y_test_true_labels = np.argmax(y_test, axis=1)
    y_test_pred_labels = np.argmax(y_test_pred, axis=1)
    
    # Calculate detailed metrics
    val_f1 = f1_score(y_val_true_labels, y_val_pred_labels, average='weighted')
    val_precision = precision_score(y_val_true_labels, y_val_pred_labels, average='weighted', zero_division=0)
    val_recall = recall_score(y_val_true_labels, y_val_pred_labels, average='weighted')
    
    test_f1 = f1_score(y_test_true_labels, y_test_pred_labels, average='weighted')
    test_precision = precision_score(y_test_true_labels, y_test_pred_labels, average='weighted', zero_division=0)
    test_recall = recall_score(y_test_true_labels, y_test_pred_labels, average='weighted')
    
    # Print results
    print(f"\n{model_name} Training Results:")
    print(f"{'='*50}")
    print(f"Validation Metrics:")
    print(f"  Accuracy:  {val_acc:.4f}")
    print(f"  F1 Score:  {val_f1:.4f}")
    print(f"  Precision: {val_precision:.4f}")
    print(f"  Recall:    {val_recall:.4f}")
    print(f"Test Metrics:")
    print(f"  Accuracy:  {test_acc:.4f}")
    print(f"  F1 Score:  {test_f1:.4f}")
    print(f"  Precision: {test_precision:.4f}")
    print(f"  Recall:    {test_recall:.4f}")
    
    metrics = {
        'val_accuracy': val_acc,
        'val_f1': val_f1,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'test_accuracy': test_acc,
        'test_f1': test_f1,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'val_predictions': y_val_pred,
        'test_predictions': y_test_pred,
        'val_true_labels': y_val_true_labels,
        'test_true_labels': y_test_true_labels
    }
    
    return model, history, metrics

print("✅ Single fold training function defined")

✅ Single fold training function defined


In [7]:
# =============================================================================
# Cell 7: Train DenseNet169 Model
# =============================================================================
print("Starting DenseNet169 training...")

# Train the model
model, history, metrics = train_single_fold_model(
    'DenseNet169', build_densenet169,
    x_train, y_train, x_val, y_val, x_test, y_test
)

print("✅ DenseNet169 training completed successfully")

Starting DenseNet169 training...

Training DenseNet169 - Single Fold
Training samples: 5776
Validation samples: 1444
Test samples: 733


I0000 00:00:1756883611.100232      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model built with 14,352,965 parameters
Steps per epoch: 120, Batch size: 24
Epoch 1/50


I0000 00:00:1756883687.179056      98 service.cc:148] XLA service 0x7c3b98003e70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756883687.179896      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1756883687.212695      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1756883687.326312      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


120/120 ━━━━━━━━━━━━━━━━━━━━ 352s 557ms/step - accuracy: 0.3167 - loss: 1.7668 - val_accuracy: 0.5048 - val_loss: 1.1566 - learning_rate: 5.0000e-05
Epoch 2/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 33s 273ms/step - accuracy: 0.5238 - loss: 1.2863 - val_accuracy: 0.5727 - val_loss: 1.0551 - learning_rate: 5.0000e-05
Epoch 3/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 32s 269ms/step - accuracy: 0.5814 - loss: 1.1467 - val_accuracy: 0.6766 - val_loss: 0.8277 - learning_rate: 5.0000e-05
Epoch 4/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 261ms/step - accuracy: 0.6133 - loss: 1.0674 - val_accuracy: 0.7348 - val_loss: 0.6544 - learning_rate: 5.0000e-05
Epoch 5/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 256ms/step - accuracy: 0.6501 - loss: 0.9971 - val_accuracy: 0.7618 - val_loss: 0.5826 - learning_rate: 5.0000e-05
Epoch 6/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 30s 251ms/step - accuracy: 0.6906 - loss: 0.9315 - val_accuracy: 0.7632 - val_loss: 0.6111 - learning_rate: 5.0000e-05
Epoch 7/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 30s 250ms/step -

In [34]:
# =============================================================================
# Grad-CAM Visualization Script for Diabetic Retinopathy Models
# FULLY REWRITTEN: Avoids Sequential model issues by using Functional API
# =============================================================================

print("="*80)
print("GRAD-CAM VISUALIZATION SCRIPT - FUNCTIONAL API VERSION")
print("="*80)

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
import os

# =============================================================================
# Define Class Labels
# =============================================================================

CLASS_LABELS = {
    0: 'No DR',
    1: 'Mild',
    2: 'Moderate', 
    3: 'Severe',
    4: 'Proliferative DR'
}

print("✅ Class labels defined:")
for idx, label in CLASS_LABELS.items():
    print(f"   {idx}: {label}")

# =============================================================================
# Cell 1: Grad-CAM Implementation (FUNCTIONAL API APPROACH)
# =============================================================================

def get_last_conv_layer_name(model):
    """
    Automatically find the last convolutional layer in the model
    """
    # Check main model layers first
    for layer in reversed(model.layers):
        if 'Conv' in layer.__class__.__name__:
            return layer.name
    
    # Check inside base model if exists
    if hasattr(model.layers[0], 'layers'):
        for layer in reversed(model.layers[0].layers):
            if 'Conv' in layer.__class__.__name__:
                return layer.name
    
    raise ValueError("Could not find convolutional layer.")


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Generate Grad-CAM heatmap using direct layer access (NO Sequential issues)
    This approach extracts the base model and accesses layers directly
    """
    # Get the base model (assumes model structure: Sequential([base_model, other_layers]))
    if hasattr(model.layers[0], 'layers'):
        base_model = model.layers[0]
    else:
        base_model = model
    
    # Get the target conv layer
    try:
        conv_layer = base_model.get_layer(last_conv_layer_name)
    except:
        # Fallback if layer not in base model
        conv_layer = model.get_layer(last_conv_layer_name)
    
    # Create a new functional model: input -> [conv_output, final_predictions]
    # This completely bypasses the Sequential model structure
    try:
        # Try to create model using base model structure
        grad_model = Model(
            inputs=base_model.input,
            outputs=[conv_layer.output, base_model.output]
        )
        use_base = True
    except:
        # Fallback to full model
        grad_model = Model(
            inputs=model.input,
            outputs=[conv_layer.output, model.output]
        )
        use_base = False
    
    # Compute gradients
    with tf.GradientTape() as tape:
        # Forward pass through grad_model
        if use_base:
            # Need to complete the forward pass through remaining layers
            conv_outputs, base_predictions = grad_model(img_array)
            
            # Pass through remaining model layers (after base_model)
            x = base_predictions
            for layer in model.layers[1:]:
                x = layer(x)
            predictions = x
        else:
            conv_outputs, predictions = grad_model(img_array)
        
        # Get predicted class
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        
        # Get loss for target class
        class_channel = predictions[:, pred_index]
    
    # Compute gradients
    grads = tape.gradient(class_channel, conv_outputs)
    
    if grads is None:
        raise ValueError(f"Gradients are None for layer {last_conv_layer_name}")
    
    # Global average pooling
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight channels and sum
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    # Normalize
    heatmap = tf.maximum(heatmap, 0)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-10)
    
    return heatmap.numpy()


def overlay_gradcam(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    """
    Overlay Grad-CAM heatmap on original image
    """
    # Resize heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # Convert image to uint8
    img_uint8 = np.uint8(255 * img)
    
    # Superimpose
    superimposed_img = heatmap * alpha + img_uint8
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')
    
    return superimposed_img


# =============================================================================
# Cell 2: Select 50 Images
# =============================================================================

def select_images_per_class(x_data, y_data, num_per_class=10, random_seed=42):
    """
    Select specified number of images per class
    """
    np.random.seed(random_seed)
    
    # Convert one-hot to integer if needed
    if len(y_data.shape) > 1:
        labels = np.argmax(y_data, axis=1)
    else:
        labels = y_data
    
    selected_indices = []
    
    for class_idx in range(5):
        class_indices = np.where(labels == class_idx)[0]
        
        if len(class_indices) >= num_per_class:
            selected = np.random.choice(class_indices, num_per_class, replace=False)
        else:
            selected = class_indices
            print(f"Warning: Only {len(class_indices)} images for class {class_idx}")
        
        selected_indices.extend(selected)
    
    selected_indices = np.array(selected_indices)
    selected_images = x_data[selected_indices]
    selected_labels = labels[selected_indices]
    
    return selected_indices, selected_images, selected_labels


print("\n📊 Selecting 50 images (10 per class)...")
selected_indices, selected_images, selected_labels = select_images_per_class(
    x_test, y_test, num_per_class=10, random_seed=42
)

print(f"✅ Selected {len(selected_indices)} images")
print(f"   Class distribution: {np.bincount(selected_labels)}")


# =============================================================================
# Cell 3: Model Configuration
# =============================================================================

MODEL_PATHS = {
    'EfficientNetV2M': '/kaggle/working/effnetv2m_final.h5',
    'DenseNet169': '/kaggle/working/densenet169_best.h5',
    'InceptionV3': '/kaggle/working/inceptionv3_final.h5',
    'EfficientNetB5': '/kaggle/working/effnetb5_final.h5'
}

# Check available models
available_models = {}
for model_name, model_path in MODEL_PATHS.items():
    if os.path.exists(model_path):
        available_models[model_name] = model_path
        print(f"✅ Found: {model_name}")
    else:
        print(f"⚠️  Not found: {model_name}")

if len(available_models) == 0:
    print("\n❌ No models found!")
else:
    print(f"\n✅ Found {len(available_models)} model(s)")


# =============================================================================
# Cell 4: Generate Grad-CAM
# =============================================================================

def generate_gradcam_for_model(model_name, model_path, images, labels):
    """
    Generate Grad-CAM visualizations
    """
    print(f"\n{'='*80}")
    print(f"Processing: {model_name}")
    print(f"{'='*80}")
    
    # Load model
    print(f"Loading model...")
    model = load_model(model_path)
    print(f"   ✓ Model loaded")
    
    # Print model structure for debugging
    print(f"   Model type: {type(model)}")
    print(f"   Number of layers: {len(model.layers)}")
    if hasattr(model.layers[0], 'layers'):
        print(f"   Base model: {type(model.layers[0])}")
        print(f"   Base model layers: {len(model.layers[0].layers)}")
    
    # Find conv layer
    try:
        last_conv_layer = get_last_conv_layer_name(model)
        print(f"✅ Using layer: {last_conv_layer}")
    except ValueError as e:
        print(f"❌ Error: {e}")
        return
    
    # Create directories
    output_dir = f'gradcam_{model_name.lower()}'
    original_dir = f'{output_dir}/original'
    overlay_dir = f'{output_dir}/overlay'
    combined_dir = f'{output_dir}/combined'
    
    os.makedirs(original_dir, exist_ok=True)
    os.makedirs(overlay_dir, exist_ok=True)
    os.makedirs(combined_dir, exist_ok=True)
    
    # Generate visualizations
    print(f"\n🔥 Generating Grad-CAM...")
    
    success_count = 0
    error_count = 0
    
    for idx, (img, true_label) in enumerate(zip(images, labels)):
        try:
            # Prepare image
            img_array = np.expand_dims(img, axis=0)
            
            # Get prediction
            preds = model.predict(img_array, verbose=0)
            pred_label = np.argmax(preds[0])
            confidence = preds[0][pred_label]
            
            # Generate Grad-CAM
            heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer, pred_index=pred_label)
            gradcam_img = overlay_gradcam(img, heatmap, alpha=0.4)
            
            # Filename
            base_filename = f'{idx+1:02d}_class{true_label}_pred{pred_label}'
            
            # Save original
            fig_orig = plt.figure(figsize=(5, 5))
            plt.imshow(img)
            plt.axis('off')
            plt.tight_layout(pad=0)
            plt.savefig(f'{original_dir}/{base_filename}_original.png', 
                       dpi=150, bbox_inches='tight', pad_inches=0)
            plt.close(fig_orig)
            
            # Save overlay
            fig_overlay = plt.figure(figsize=(5, 5))
            plt.imshow(gradcam_img)
            plt.axis('off')
            plt.tight_layout(pad=0)
            plt.savefig(f'{overlay_dir}/{base_filename}_overlay.png', 
                       dpi=150, bbox_inches='tight', pad_inches=0)
            plt.close(fig_overlay)
            
            # Combined visualization
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            axes[0].imshow(img)
            axes[0].set_title(f'Original\nTrue: {CLASS_LABELS[true_label]}', fontsize=10)
            axes[0].axis('off')
            
            axes[1].imshow(heatmap, cmap='jet')
            axes[1].set_title('Grad-CAM', fontsize=10)
            axes[1].axis('off')
            
            axes[2].imshow(gradcam_img)
            axes[2].set_title(f'Overlay\nPred: {CLASS_LABELS[pred_label]} ({confidence:.2%})', fontsize=10)
            axes[2].axis('off')
            
            correct = "✓" if true_label == pred_label else "✗"
            fig.suptitle(f'{model_name} - Image {idx+1}/50 {correct}', 
                        fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            plt.savefig(f'{combined_dir}/{base_filename}_combined.png', 
                       dpi=150, bbox_inches='tight')
            plt.close(fig)
            
            success_count += 1
            
            if (idx + 1) % 10 == 0:
                print(f"   ✓ Processed {idx+1}/50 ({success_count} successful)")
                
        except Exception as e:
            error_count += 1
            print(f"   ⚠️ Error on image {idx+1}: {str(e)}")
            continue
    
    print(f"\n✅ Completed: {success_count} successful, {error_count} errors")
    print(f"   Output: '{output_dir}/'")
    
    # Cleanup
    del model
    tf.keras.backend.clear_session()


# Process all models
for model_name, model_path in available_models.items():
    generate_gradcam_for_model(model_name, model_path, selected_images, selected_labels)


# =============================================================================
# Cell 5: Summary Comparison
# =============================================================================

print("\n" + "="*80)
print("CREATING SUMMARY COMPARISON")
print("="*80)

def create_summary_comparison(models_dict, images, labels, num_samples=5):
    """
    Create model comparison visualizations
    """
    print(f"\nCreating comparison for {num_samples} samples...")
    
    np.random.seed(42)
    
    # Select samples
    sample_indices = []
    for class_idx in range(min(5, num_samples)):
        class_mask = labels == class_idx
        class_imgs = np.where(class_mask)[0]
        if len(class_imgs) > 0:
            sample_indices.append(np.random.choice(class_imgs))
    
    # Load models
    loaded_models = {}
    conv_layers = {}
    
    print("Loading models...")
    for model_name, model_path in models_dict.items():
        loaded_models[model_name] = load_model(model_path)
        conv_layers[model_name] = get_last_conv_layer_name(loaded_models[model_name])
        print(f"   ✓ {model_name}")
    
    os.makedirs('gradcam_comparison', exist_ok=True)
    
    for sample_idx, img_idx in enumerate(sample_indices):
        print(f"   Comparison {sample_idx+1}/{len(sample_indices)}...")
        
        img = images[img_idx]
        true_label = labels[img_idx]
        
        num_models = len(loaded_models)
        fig, axes = plt.subplots(1, num_models + 1, figsize=(5 * (num_models + 1), 5))
        
        # Original
        axes[0].imshow(img)
        axes[0].set_title(f'Original\nTrue: {CLASS_LABELS[true_label]}', 
                         fontsize=11, fontweight='bold')
        axes[0].axis('off')
        
        # Grad-CAM from each model
        img_array = np.expand_dims(img, axis=0)
        
        for idx, (model_name, model) in enumerate(loaded_models.items()):
            preds = model.predict(img_array, verbose=0)
            pred_label = np.argmax(preds[0])
            confidence = preds[0][pred_label]
            
            heatmap = make_gradcam_heatmap(img_array, model, conv_layers[model_name], pred_index=pred_label)
            gradcam_img = overlay_gradcam(img, heatmap, alpha=0.4)
            
            axes[idx + 1].imshow(gradcam_img)
            axes[idx + 1].set_title(f'{model_name}\n{CLASS_LABELS[pred_label]} ({confidence:.1%})', 
                                   fontsize=11, fontweight='bold')
            axes[idx + 1].axis('off')
        
        fig.suptitle(f'Model Comparison - Sample {sample_idx + 1} (Class: {CLASS_LABELS[true_label]})', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        plt.savefig(f'gradcam_comparison/comparison_sample_{sample_idx + 1}.png', 
                   dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"✅ Saved to 'gradcam_comparison/'")
    
    # Cleanup
    for model in loaded_models.values():
        del model
    tf.keras.backend.clear_session()


# Create comparison
if len(available_models) > 1:
    create_summary_comparison(available_models, selected_images, selected_labels, num_samples=5)
elif len(available_models) == 1:
    print("⚠️  Only 1 model. Skipping comparison.")

print("\n" + "="*80)
print("✅ GRAD-CAM COMPLETED!")
print("="*80)
print(f"\n📁 Output directories:")
for model_name in available_models.keys():
    model_dir = f"gradcam_{model_name.lower()}"
    print(f"   - {model_dir}/")
if len(available_models) > 1:
    print(f"   - gradcam_comparison/")
print("\n🎉 Done!")


GRAD-CAM VISUALIZATION SCRIPT - FUNCTIONAL API VERSION
✅ Class labels defined:
   0: No DR
   1: Mild
   2: Moderate
   3: Severe
   4: Proliferative DR

📊 Selecting 50 images (10 per class)...
✅ Selected 50 images
   Class distribution: [10 10 10 10 10]
⚠️  Not found: EfficientNetV2M
✅ Found: DenseNet169
⚠️  Not found: InceptionV3
⚠️  Not found: EfficientNetB5

✅ Found 1 model(s)

Processing: DenseNet169
Loading model...
   ✓ Model loaded
   Model type: <class 'keras.src.models.sequential.Sequential'>
   Number of layers: 6
   Base model: <class 'keras.src.models.functional.Functional'>
   Base model layers: 595
✅ Using layer: conv5_block32_2_conv

🔥 Generating Grad-CAM...
   ✓ Processed 10/50 (10 successful)
   ✓ Processed 20/50 (20 successful)
   ✓ Processed 30/50 (30 successful)
   ✓ Processed 40/50 (40 successful)
   ✓ Processed 50/50 (50 successful)

✅ Completed: 50 successful, 0 errors
   Output: 'gradcam_densenet169/'

CREATING SUMMARY COMPARISON
⚠️  Only 1 model. Skipping comp